# Sandwich Shop Simulation
Here we look at the case of a restaurant modeling its operations.  We can
see how the different parameter settings affect the average profit.

In [2]:
# Import necessary packages
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import time

## Sandwich Shop Parameters
* A sandwich shop orders $O$ sandwiches each day
* At a fixed price of $W$ dollars.  
* Sell price of $R$ dollars.  
* The demand for sandwiches, $D$, is a random variable that is $\mathcal{U}\{5, 6, \ldots, 35\}$.  

How much profit will be made on average? We may simulate $n$ replications of $4$ consecutive days:


In [52]:
whole = 1  # wholesale price of sandwich
retail = 5 # retail price of sandwich
order = 20 # quantity of sandwiches ordered daily
demandlo = 5 # lo end of demand
demandhi = 35 # hi end of demand
ndays = 3*365 # number of days for simulation
ndvec = range(1,ndays+1)  # vector of 1 to number of days
nreps = 10000 # number of replications
nrvec = range(1,nreps+1) # number of replications of the simulation

## Perform Simulation
* The number of sandwiches sold, $S$, is the minimum of the demand, and the number ordered
$$S = \min (D,O).$$
* The profit, $P$ each day is the difference between the income and the expense:
$$P = S \times R - O \times W$$

In our case we are computing $P_{ij}$ for $i=1, \ldots, n$ and $ = 1, \ldots, m$.  Then we compute the average profit of $m$ days for each run $i$

In [53]:
wholeprofit = 0
for i in range(nreps):
    for j in range(ndays):
        demand = rd.randint(demandlo,demandhi)
        sold = np.min([demand,order])
        dayprofit = sold*retail-sold*whole
        wholeprofit = dayprofit + wholeprofit
avgprofit = wholeprofit/nreps/ndays


In [50]:
demand = np.random.randint(demandlo,demandhi,[nreps,ndays]) # uniform random numbers for demand
sold = np.minimum(demand,order) # amount of sandwiches sold that day
dayprofit = sold*retail-order*whole # profit for the day
avgprofitrun = np.mean(dayprofit, axis = 0) # average profit for the first m days
avgprofit= np.mean(dayprofit)  # average profit over all runs and days

In [51]:
print("For", nreps, "replications of", ndays, "days of business, \n",
      "sandwiches costing", whole, " a piece and sold for $",retail, "a piece.") 
print("For a supply of ", order, " sandwiches ordered daily and a random demand\n"
       "that is uniform over a range of {", demandlo, ",",demandhi,"} sandwiches" )
print("The average daily profit over this whole time = $", avgprofit, "\n"
      "compared to the maximum possible profit of $",order*(retail-whole))


For 10000 replications of 1095 days of business, 
 sandwiches costing 1  a piece and sold for $ 5 a piece.
For a supply of  25  sandwiches ordered daily and a random demand
that is uniform over a range of { 5 , 35 } sandwiches
The average daily profit over this whole time = $ 64.98500182648402 
compared to the maximum possible profit of $ 100
